In [ ]:
!pip install rouge-score --quiet

  Preparing metadata (setup.py) ... done


In [ ]:
!pip install bert_score --quiet # Install the bert_score package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.9 MB/s eta 0:00:00


In [ ]:
pip install langchain python-docx pdfplumber nltk --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 18.3 MB/s eta 0:00:00


- **langchain**: To manage chunking and embedding.
- **python-docx**: For handling Word documents.
- **pdfplumber**: For extracting text from PDFs.
- **nltk**: For tokenization.


In [ ]:
!pip install faiss-cpu --quiet


In [ ]:
!pip install faiss-cpu transformers sentence-transformers --quiet

In [ ]:
!pip install sentence-transformers --quiet

In [ ]:
!pip install -U langchain-community --quiet

The **langchain-community** package provides additional modules and integrations for LangChain.

In [ ]:
!pip install tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.4 MB/s eta 0:00:00


In [ ]:
import os
import pdfplumber
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from nltk.tokenize import sent_tokenize

# Function to read text from a .txt file
def read_txt(file_path):
    """Reads text from a .txt file."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return f.read()

# Function to read text from a PDF file
def read_pdf(file_path):
    """Reads text from a PDF file."""
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
    return text

# Function to read text from a Word (.docx) file
def read_docx(file_path):
    """Reads text from a Word (.docx) file."""
    doc = Document(file_path)
    return "\n".join([para.text for para in doc.paragraphs])

def chunk_text(text, chunk_size=500, overlap=50):
    """
    Splits text into smaller chunks of a specified size.
    Uses RecursiveCharacterTextSplitter for efficient splitting.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=overlap,
        separators=["\n\n", "\n", ".", " ", ""]
    )
    chunks = text_splitter.split_text(text)
    return chunks

if __name__ == "__main__":
    # Step 1: Specify the file path containing your document
    file_path = "/content/depression.pdf"  # Changed to file path

    # Step 2: Collect text data from the file
    # Directly read the file using the correct function
    if file_path.endswith('.txt'):
        collected_text = read_txt(file_path)
    elif file_path.endswith('.pdf'):
        collected_text = read_pdf(file_path)
    elif file_path.endswith('.docx'):
        collected_text = read_docx(file_path)
    else:
        collected_text = ""
        print("Unsupported file type")

    print("Collected Data: ", collected_text[:500])  # Print the first 500 characters of the collected text

    # Step 3: Chunk the collected text
    chunks = chunk_text(collected_text, chunk_size=500, overlap=50)
    print("\nNumber of Chunks: ", len(chunks))  # Print the number of chunks
    print("\nFirst Chunk:\n", chunks[0])  # Print the first chunk

    # Step 5: Save the chunks to a file
    with open("chunks.txt", "w", encoding="utf-8") as f:
        for chunk in chunks:
            f.write(chunk + "\n---\n")
    print("\nChunks saved to 'chunks.txt'")


Collected Data:  Depression
National Institute
of Mental Health
What is depression?
Everyone feels sad or low sometimes, but these feelings usually pass. Depression (also
called major depression, major depressive disorder, or clinical depression) is different.
It can cause severe symptoms that affect how a person feels, thinks, and handles daily
activities, such as sleeping, eating, or working.
Depression can affect anyone regardless of age, gender, race or ethnicity, income,
culture, or education. Research sugg

Number of Chunks:  40

First Chunk:
 Depression
National Institute
of Mental Health
What is depression?
Everyone feels sad or low sometimes, but these feelings usually pass. Depression (also
called major depression, major depressive disorder, or clinical depression) is different.
It can cause severe symptoms that affect how a person feels, thinks, and handles daily
activities, such as sleeping, eating, or working.
Depression can affect anyone regardless of age, gender, race or

**Step 3: Chunk Data**

Chunking is essential to ensure that the documents fit within token limits of the LLM and are more manageable for retrieval.

Large documents can be challenging for retrieval and LLMs to process. So, split the data into smaller, manageable chunks (e.g., 500–1000 tokens).



**Why chunk the data?**
- LLMs have token limits.
- Smaller chunks improve retrieval accuracy.
- Example tools: LangChain and LlamaIndex can automate data chunking.


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Initialize the SentenceTransformer model
# You can choose a model from Hugging Face's model hub
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and efficient

# Example text chunks
# text_chunks = [
#     "Once upon a time, there was a king.",
#     "The king ruled over a vast and beautiful kingdom.",
#     "He had three daughters who were the pride of his heart."
# ]

# Generate embeddings for each chunk
embeddings = model.encode(chunks)

# Display embeddings
for i, embedding in enumerate(embeddings):
    print(f"Chunk {i+1} Embedding (Shape: {embedding.shape}):\n{embedding}\n")

# Save embeddings for later use (Optional)
np.save("text_embeddings.npy", embeddings)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Chunk 1 Embedding (Shape: (384,)):
[ 4.63483073e-02  3.61155346e-02  4.27635983e-02  5.79926260e-02
  1.21611757e-02  2.90139150e-02 -2.85976660e-03  7.41440728e-02
  4.54266630e-02  3.62166623e-03 -8.01777765e-02 -1.74724832e-02
 -5.15295267e-02 -2.21621916e-02  5.94629049e-02 -1.60023589e-02
 -2.30011586e-02 -1.13529647e-02 -3.50403003e-02  3.59337963e-02
 -3.76856024e-03  1.24357762e-02 -2.29259748e-02  6.84606582e-02
 -8.24795440e-02  8.52104127e-02 -5.61389327e-03 -5.32309227e-02
  9.20375157e-03  5.65910786e-02  4.04238403e-02  8.18982050e-02
  8.56449232e-02  9.50041190e-02  4.12793122e-02  2.46875435e-02
 -5.28724827e-02  7.58424327e-02  3.33100967e-02 -7.78973242e-03
 -3.24111730e-02  5.45409955e-02  5.76058663e-02  3.40106003e-02
  1.19117377e-02 -5.20139188e-02 -4.80114929e-02  2.10160017e-02
 -1.17860944e-03 -3.12551297e-02  5.06970398e-02  2.73951646e-02
 -2.77455593e-03 -1.17687667e-02  4.56749052e-02 -2.54620984e-02
  4.01401445e-02 -5.57431346e-03  2.79825856e-03  7.403

In [ ]:
# Save embeddings to a .npy file
np.save("embeddings.npy", embeddings)

# Load embeddings from the .npy file
loaded_embeddings = np.load("embeddings.npy")

print("Loaded Embeddings:\n", loaded_embeddings)


Loaded Embeddings:
 [[ 0.04634831  0.03611553  0.0427636  ... -0.00957096  0.03066018
   0.00500197]
 [ 0.06084532  0.0780421  -0.02099289 ...  0.05420969  0.0843902
  -0.04503496]
 [ 0.06771979  0.00203123  0.03991065 ... -0.00998218  0.04804375
  -0.01648244]
 ...
 [ 0.04974539 -0.01931047 -0.02389374 ... -0.01943531  0.05260934
   0.05621746]
 [ 0.1023833   0.02672691 -0.03375961 ...  0.03596023 -0.05858976
  -0.0164011 ]
 [ 0.01015527  0.00423791 -0.05387904 ...  0.00470629  0.02513501
   0.04006247]]


**Key Features of This Code**
1. Model Choice:

 -  all-MiniLM-L6-v2: A lightweight and fast model with good accuracy.
 - Other options include all-MPNet-V2, paraphrase-MiniLM-L6-v2, etc. You can browse more models on Hugging Face Sentence Transformers.
Embeddings Shape:

**How Does the Model Use These Vectors?**
1. Text Encoding:

 - The model takes a sentence or text chunk as input.
 - It generates a 384-dimensional embedding vector where similar sentences are close to each other in this high-dimensional space.
2. Retrieval:

 - When a query is made, its embedding vector is generated.
 - The query embedding is compared to the embeddings of the document chunks using similarity measures (like cosine similarity).
 - Chunks with the highest similarity are retrieved as the most relevant.
3. Generation:

 - The retrieved chunks are fed to the language model to generate a contextually accurate answer.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

In [ ]:
import faiss
import numpy as np

# Convert embeddings to a NumPy array
embeddings_array = np.array(embeddings).astype('float32')

# Create a FAISS index
dimension = embeddings_array.shape[1]  # Embedding size (e.g., 1536 for OpenAI models)
faiss_index = faiss.IndexFlatL2(dimension)

# Add embeddings to the index
faiss_index.add(embeddings_array)

print("FAISS index created and embeddings added.")


FAISS index created and embeddings added.


In [ ]:
faiss.write_index(faiss_index, "faiss_index.bin")


In [ ]:
# Query embedding (e.g., for a user query)
query = "What is depression?"
query_embedding = model.encode(query) # Generate embedding for query
# embeddings = model.encode(chunks) # This is not used here and not required

# Perform search
top_k = 3  # Number of results to retrieve
distances, indices = faiss_index.search(np.array([query_embedding]).astype('float32'), top_k)

print("Top matching chunks:")
for i in indices[0]:
    print(chunks[i])

Top matching chunks:
Depression
National Institute
of Mental Health
What is depression?
Everyone feels sad or low sometimes, but these feelings usually pass. Depression (also
called major depression, major depressive disorder, or clinical depression) is different.
It can cause severe symptoms that affect how a person feels, thinks, and handles daily
activities, such as sleeping, eating, or working.
Depression can affect anyone regardless of age, gender, race or ethnicity, income,
culture, or education. Research suggests that genetic, biological, environmental, and
psychological factors play a role in the disorder.
Women are diagnosed with depression more often than men, but men can also be
depressed. Because men may be less likely to recognize, talk about, and seek help for
their negative feelings, they are at greater risk of their depression symptoms being
undiagnosed and undertreated. Studies also show higher rates of depression and an
sadness, or loss of enjoyment.
What are the sign

**Step 2: Index the Chunks into a Vector Database**

FAISS (Facebook AI Similarity Search) is a library that allows developers to quickly search for embeddings of multimedia documents that are similar to each other . Indexing embeddings into a vector database like FAISS or Pinecone allows for efficient similarity search.

**Option 1: Using FAISS**

FAISS is a library designed for fast similarity searches. The IndexFlatL2 method is used for L2 (Euclidean) distance-based search.




In [ ]:
from transformers import pipeline
# Retrieve the most relevant text chunks
retrieved_chunks = [chunks[idx] for idx in indices[0]]
print("Relevant Chunks:", retrieved_chunks)

# Step 5: Integrate with a Language Model
# Using Hugging Face's transformers for text generation
language_model = pipeline("text-generation", model="gpt2")  # Load GPT-2
prompt = f"Question: {query}\nContext: {' '.join(retrieved_chunks)}\nAnswer:"
response = language_model(prompt, max_length=500, num_return_sequences=1)

# Output the generated answer
print("Generated Answer:", response[0]['generated_text'])

Relevant Chunks: ['Depression\nNational Institute\nof Mental Health\nWhat is depression?\nEveryone feels sad or low sometimes, but these feelings usually pass. Depression (also\ncalled major depression, major depressive disorder, or clinical depression) is different.\nIt can cause severe symptoms that affect how a person feels, thinks, and handles daily\nactivities, such as sleeping, eating, or working.\nDepression can affect anyone regardless of age, gender, race or ethnicity, income,', 'culture, or education. Research suggests that genetic, biological, environmental, and\npsychological factors play a role in the disorder.\nWomen are diagnosed with depression more often than men, but men can also be\ndepressed. Because men may be less likely to recognize, talk about, and seek help for\ntheir negative feelings, they are at greater risk of their depression symptoms being\nundiagnosed and undertreated. Studies also show higher rates of depression and an', 'sadness, or loss of enjoyment.\

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Generated Answer: Question: What is depression?
Context: Depression
National Institute
of Mental Health
What is depression?
Everyone feels sad or low sometimes, but these feelings usually pass. Depression (also
called major depression, major depressive disorder, or clinical depression) is different.
It can cause severe symptoms that affect how a person feels, thinks, and handles daily
activities, such as sleeping, eating, or working.
Depression can affect anyone regardless of age, gender, race or ethnicity, income, culture, or education. Research suggests that genetic, biological, environmental, and
psychological factors play a role in the disorder.
Women are diagnosed with depression more often than men, but men can also be
depressed. Because men may be less likely to recognize, talk about, and seek help for
their negative feelings, they are at greater risk of their depression symptoms being
undiagnosed and undertreated. Studies also show higher rates of depression and an sadness, or 

**Step 5: Integrate with a Language Model**
- The relevant chunks are concatenated and passed as context to a language model (e.g., GPT-2 from Hugging Face's transformers).
- A prompt is constructed containing the query and the retrieved context, and the language model generates a response.


In [ ]:
from transformers import pipeline

# Example query and retrieved chunks
query = "What is depression?"
# Concatenate the retrieved chunks as context
context = " ".join(retrieved_chunks)

# Initialize the summarization pipeline using facebook/bart-large-cnn
summarization_model = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)  # Use CPU

# Prepare the input for summarization
input_text = f"Question: {query}\nContext: {context}"

# Generate the summary
summary = summarization_model(input_text, max_length=315, min_length=50, truncation=True)

# Print the generated answer
print("Generated Answer:", summary[0]['summary_text'])


Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Generated Answer: Common signs and symptoms of depression include: Persistent sad, anxious, or “empty” mood. Depression can also involve other changes in mood or behavior. Not everyone who is depressed shows all these symptoms. Some people experience only a few symptoms, while others experience many.


**Model**:

The **facebook/bart-large-cnn** model is designed for text summarization, making it ideal for generating concise and coherent answers from large contexts.


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Step 2: Query Input
print("Interactive RAG Agent is Ready!")
query = input("You: ")  # User provides a question

# Step 3: Retrieve Relevant Chunks
# Use TF-IDF Vectorizer for retrieval
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform(chunks)
query_vector = vectorizer.transform([query])

# Compute cosine similarity to find relevant chunks
similarities = cosine_similarity(query_vector, vectors).flatten()
top_indices = np.argsort(similarities)[-3:][::-1]  # Retrieve top 3 most relevant chunks
retrieved_chunks = [chunks[i] for i in top_indices]
print("\nRetrieved Chunks:", retrieved_chunks)

# Step 4: Generate Answer Using a Language Model
# Use facebook/bart-large-cnn for summarization/generation
print("\nGenerating Answer...")
model_name = "facebook/bart-large-cnn"  # Change this to another model if needed
summarization_model = pipeline("summarization", model=model_name)

# Combine query and retrieved chunks as input
context = " ".join(retrieved_chunks)
input_text = f"Question: {query}\nContext: {context}"

# Generate the answer
summary = summarization_model(input_text, max_length=150, min_length=50, truncation=True)
print("\nRAG Agent:", summary[0]['summary_text'])


Interactive RAG Agent is Ready!
You: How is depression diagnosed?

Retrieved Chunks: ['threatening situations, call 911.\nHow is depression diagnosed?\nTo be diagnosed with depression, a person must have symptoms most of the day,\nnearly every day, for at least 2 weeks. One of the symptoms must be a depressed\nmood or a loss of interest or pleasure in most activities. Children and adolescents may\nbe irritable rather than sad.\nAlthough several persistent symptoms, in addition to low mood, are required for a depression', 'Depression\nNational Institute\nof Mental Health\nWhat is depression?\nEveryone feels sad or low sometimes, but these feelings usually pass. Depression (also\ncalled major depression, major depressive disorder, or clinical depression) is different.\nIt can cause severe symptoms that affect how a person feels, thinks, and handles daily\nactivities, such as sleeping, eating, or working.\nDepression can affect anyone regardless of age, gender, race or ethnicity, income,'

Device set to use cpu
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



RAG Agent: To be diagnosed with depression, a person must have symptoms most of the day, nearly every day, for at least 2 weeks. One of the symptoms must be a depressed mood or a loss of interest or pleasure in most activities. Depression can affect anyone regardless of age, gender, race or ethnicity.


**BLEU (Bilingual Evaluation Understudy)**:
- Measures the n-gram overlap between the generated text and a reference text.
- Calculates the precision of the generated text with respect to the reference text.
- Higher scores indicate better similarity between the generated and reference texts.
- Commonly used for evaluating machine translation and text summarization systems.

**ROUGE (Recall-Oriented Understudy for Gisting Evaluation)**:
- Evaluates the recall of n-grams and sequences between the generated text and a reference text.
- ROUGE-N measures the recall of n-grams, while ROUGE-L measures the recall of longest contiguous sequences.
- ROUGE is commonly used for evaluating text summarization and machine translation systems.


**BERTScore**:
- Uses contextualized embeddings from BERT (Bidirectional Encoder Representations from Transformers) to measure semantic similarity between the generated text and a reference text.
- Calculates the cosine similarity between the embeddings of the generated text and the reference text.
- BERTScore is commonly used for evaluating text summarization, machine translation, and text generation systems.


**Note** that BERTScore is a more nuanced metric that captures the semantic meaning of the text, whereas BLEU and ROUGE focus on surface-level n-gram matching.


In [ ]:
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu
from bert_score import score as bert_score

# Example generated and reference answers
generated_answers = [
    "To be diagnosed with depression, a person must have symptoms most of the day, nearly every day, for at least 2 weeks. One of the symptoms must be a depressed mood or a loss of interest or pleasure in most activities. Depression can affect anyone regardless of age, gender, race or ethnicity."
]
reference_answers = [
     "threatening situations, call 911.\nHow is depression diagnosed?\nTo be diagnosed with depression, a person must have symptoms most of the day,\nnearly every day, for at least 2 weeks. One of the symptoms must be a depressed\nmood or a loss of interest or pleasure in most activities. Children and adolescents may\nbe irritable rather than sad.\nAlthough several persistent symptoms, in addition to low mood, are required for a depression', 'Depression\nNational Institute\nof Mental Health\nWhat is depression?\nEveryone feels sad or low sometimes, but these feelings usually pass. Depression (also\ncalled major depression, major depressive disorder, or clinical depression) is different.\nIt can cause severe symptoms that affect how a person feels, thinks, and handles daily\nactivities, such as sleeping, eating, or working.\nDepression can affect anyone regardless of age, gender, race or ethnicity, income,', 'diagnosis, people with only a few symptoms may benefit from treatment. The severity\nand frequency of symptoms and how long they last vary depending on the person.\nIf you think you may have depression, talk to a health care provider, such as a primary\ncare doctor, psychologist, or psychiatrist. During the visit, the provider may ask when\nyour symptoms began, how long they have lasted, how often they occur, and if they"
        ]

# Evaluate using ROUGE
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
for gen, ref in zip(generated_answers, reference_answers):
    print(rouge.score(gen, ref))

# Evaluate using BLEU
for gen, ref in zip(generated_answers, reference_answers):
    print("BLEU:", sentence_bleu([ref.split()], gen.split()))

# Evaluate using BERTScore
P, R, F1 = bert_score(generated_answers, reference_answers, lang="en")
print(f"BERTScore - Precision: {P.mean()}, Recall: {R.mean()}, F1: {F1.mean()}")


{'rouge1': Score(precision=0.24644549763033174, recall=1.0, fmeasure=0.3954372623574145), 'rouge2': Score(precision=0.23809523809523808, recall=0.9803921568627451, fmeasure=0.3831417624521073), 'rougeL': Score(precision=0.24644549763033174, recall=1.0, fmeasure=0.3954372623574145)}
BLEU: 0.044631898355837335


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.9531250596046448, Recall: 0.8417101502418518, F1: 0.8939595222473145


**Overall Summary**

**ROUGE**: High recall shows that the generated answer includes most relevant content, but lower precision indicates some irrelevant or redundant content.

**BLEU**: Low score suggests the phrasing differs significantly from the reference answer.

**BERTScore**: High semantic similarity, showing the generated text conveys the same meaning as the reference, even with different phrasing.
